# Text Normalization (Chinese)

- `text_normalizer_zh.py`
- Including functions for:
    - word-seg chinese texts
    - clean up texts by removing duplicate spaces and line breaks
    - remove incompatible weird characters

In [1]:
import unicodedata
import re
#from nltk.corpus import wordnet
#import collections
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
import requests
import pandas as pd
import text_normalizer_zh as tnz

## Normalization Functions

In [2]:
# %load text_normalizer_zh.py
"""

Notes
-----

    These functions are based on the text normalization functions 
    provided in Text Analytics with Python 2ed.

"""

import unicodedata
import re
# from nltk.tokenize.toktok import ToktokTokenizer
import pandas as pd
import jieba

## Initialize Trad Chinese dictionary
jieba.set_dictionary('../../../RepositoryData/data/jiaba/dict.txt.jiebatw.txt')


## Normalize unicode characters
def remove_weird_chars(text):
    #     ```
    #     (NFKD) will apply the compatibility decomposition, i.e.
    #     replace all compatibility characters with their equivalents.
    #     ```
    text = unicodedata.normalize('NFKD', text).encode('utf-8',
                                                      'ignore').decode(
                                                          'utf-8', 'ignore')
    return text


## Remove extra linebreaks
def remove_extra_linebreaks(text):
    lines = text.split(r'\n+')
    return '\n'.join(
        [re.sub(r'[\s]+', ' ', l).strip() for l in lines if len(l) != 0])


## Remove extra medial/trailing/leading spaces
def remove_extra_spaces(text):
    return re.sub("\\s+", " ", text).strip()


## Seg the text into words
def seg(text):
    text_seg = jieba.cut(text)
    out = ' '.join(text_seg)
    return out


## Remove punctuation/symbols
def remove_symbols(text):
    """
    
    Unicode 6.0 has 7 character categories, and each category has subcategories:

    Letter (L): lowercase (Ll), modifier (Lm), titlecase (Lt), uppercase (Lu), other (Lo)
    Mark (M): spacing combining (Mc), enclosing (Me), non-spacing (Mn)
    Number (N): decimal digit (Nd), letter (Nl), other (No)
    Punctuation (P): connector (Pc), dash (Pd), initial quote (Pi), final quote (Pf), open (Ps), close (Pe), other (Po)
    Symbol (S): currency (Sc), modifier (Sk), math (Sm), other (So)
    Separator (Z): line (Zl), paragraph (Zp), space (Zs)
    Other (C): control (Cc), format (Cf), not assigned (Cn), private use (Co), surrogate (Cs)
    
    
    There are 3 ranges reserved for private use (Co subcategory): 
    U+E000—U+F8FF (6,400 code points), U+F0000—U+FFFFD (65,534) and U+100000—U+10FFFD (65,534). 
    Surrogates (Cs subcategory) use the range U+D800—U+DFFF (2,048 code points).
    
    
    """

    ## Brute-force version: list all possible unicode ranges, but this list is not complete.
    #   text = re.sub('[\u0021-\u002f\u003a-\u0040\u005b-\u0060\u007b-\u007e\u00a1-\u00bf\u2000-\u206f\u2013-\u204a\u20a0-\u20bf\u2100-\u214f\u2150-\u218b\u2190-\u21ff\u2200-\u22ff\u2300-\u23ff\u2460-\u24ff\u2500-\u257f\u2580-\u259f\u25a0-\u25ff\u2600-\u26ff\u2e00-\u2e7f\u3000-\u303f\ufe50-\ufe6f\ufe30-\ufe4f\ufe10-\ufe1f\uff00-\uffef─◆╱]+','',text)

    text = ''.join(ch for ch in text
                   if unicodedata.category(ch)[0] not in ['P', 'S'])
    return text


## Remove numbers
def remove_numbers(text):
    return re.sub('\\d+', "", text)


## Remove alphabets
def remove_alphabets(text):
    return re.sub('[a-zA-Z]+', '', text)


## Combine every step
def normalize_corpus(corpus,
                     is_remove_extra_linebreaks=True,
                     is_remove_weird_chars=True,
                     is_seg=True,
                     is_remove_symbols=True,
                     is_remove_numbers=True,
                     is_remove_alphabets=True):

    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:

        if is_remove_extra_linebreaks:
            doc = remove_extra_linebreaks(doc)

        if is_remove_weird_chars:
            doc = remove_weird_chars(doc)

        if is_seg:
            doc = seg(doc)

        if is_remove_symbols:
            doc = remove_symbols(doc)

        if is_remove_alphabets:
            doc = remove_alphabets(doc)

        if is_remove_numbers:
            doc = remove_numbers(doc)

        normalized_corpus.append(remove_extra_spaces(doc))

    return normalized_corpus

## Extract an Article

- Grab the first article from Google news for demonstration

In [3]:
url = 'https://news.google.com/topics/CAAqJQgKIh9DQkFTRVFvSUwyMHZNRFptTXpJU0JYcG9MVlJYS0FBUAE?hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
r = requests.get(url)
web_content = r.text
soup = BeautifulSoup(web_content, 'lxml')
title = soup.find_all('a', class_='DY5T1d')
first_art_link = title[0]['href'].replace('.', 'https://news.google.com', 1)

#print(first_art_link)
art_request = requests.get(first_art_link)
art_request.encoding = 'utf8'
soup_art = BeautifulSoup(art_request.text, 'lxml')

art_content = soup_art.find_all('p')
art_texts = [p.text for p in art_content]
print(art_texts)

['2018年10月21日，台鐵發生普悠瑪號列車翻覆事故，到今天正好屆滿2周年。據悉，交通部長林佳龍日前主動要求台鐵進行安全報告，不料竟赫然發現，今年5月19日台鐵台中成功站南側，竟因斷軌44公分卻通報不實，讓莒光號及普悠瑪號兩班列車高速通過斷軌處，險再釀重大事故。更離譜的是，該處早在2個月前已出現裂痕卻遲遲未處理，讓林佳龍當場震怒，痛斥台鐵「沒有最扯，只有更扯！」、「兩年前這麼多無辜的生命，不能白白犧牲！」', '相關新聞：台鐵App出包！旅客買了自強號看不到座位\u3000悲情一路站到目的地', '交通部回應表示，林佳龍已要求台鐵檢討，懲處名單報部核定，提高懲處層級，包括局長在內，一併全面檢討相關責任。', '林佳龍在2019年初上任交通部長後，要求針對普悠瑪列車翻覆事故重啟調查、擴大懲處、提高賠償，家屬陸續接受賠償條件。據了解，今年事故周年紀念前夕，林佳龍要求台鐵局、鐵道局、路政司等單位，向他彙報台鐵的安全策進作為，原是希望藉此激勵台鐵提高安全管理，豈料卻因得知519事故經過而大怒。', '運安會曾在今年5月30日發布「安全通告」，要求台鐵對全台的彎道鋼軌加強檢查，當時曾指出台鐵5月19日發生內軌斷裂、所幸無人傷亡，但並未對外透露事發詳細經過。交通部5月19日當天雖接獲台鐵簡訊通報有鋼軌事故，但稍晚即已緊急修復，恢復雙向通行。', '不過據《蘋果》掌握消息，林佳龍日前主動要求台鐵報告時，台鐵才首度向交通部坦言，今年5月19日晚間18時59分，台鐵第3218次司機員向成功站通報：「鐵軌碰一聲很大聲，好像是斷軌。」站務人員通知道班後共同前往查看，但竟然未依SOP向調度員通報，導致後續有兩班列車包括2次莒光號、288次普悠瑪自強號，高速通過該地點。直到調度員自行從無線電群組聽到成功站的通話，得知有鋼軌斷裂，才在19時27分發布行車命令封鎖該線，改單線雙向行車，過程相當驚險。', '隨後成功道班趕至現場緊急搶修，發現鋼軌斷裂44公分，先以1公尺短軌緊急更換，21時26分完成搶修才恢復雙線行車，當晚收班後再連夜換新鋼軌。隔天台鐵通報運安會立案調查，也才有10天之後的安全通告。', '據了解，台鐵追查後發現，成功道班領班早在今年3月3日，就發現該處鋼軌出現裂紋，發現後及通知相關人員處理，進行臨時補強措施；依標準作業程序，當晚就應該抽換新的鋼軌，不料相關人員竟置之不理，此案

- Normalized results:

In [4]:
tnz.normalize_corpus([' '.join(art_texts)])

Building prefix dict from /Users/Alvin/GoogleDrive/_MySyncDrive/RepositoryData/data/jiaba/dict.txt.jiebatw.txt ...


Dumping model to file cache /var/folders/n7/ltpzwx813c599nfxfb94s_640000gn/T/jieba.u1b52b47246a0f2e6497af6bbe107adac.cache


Loading model cost 2.027 seconds.


Prefix dict has been built successfully.


['年 月 日 台鐵 發生 普悠瑪號 列車 翻覆 事故 到 今天 正好 屆滿 周年 據悉 交通部長 林佳龍 日前 主動 要求 台鐵 進行 安全 報告 不料 竟 赫然 發現 今年 月 日 台鐵 台中 成功 站 南側 竟因 斷軌 公分 卻 通報 不實 讓 莒光號 及 普悠瑪號 兩班 列車 高速 通過 斷軌 處 險 再 釀 重大 事故 更 離譜 的 是 該處 早 在 個 月 前 已 出現 裂痕 卻 遲遲 未 處理 讓 林佳龍 當場 震怒 痛斥 台鐵 沒有 最扯 只有 更 扯 兩年 前 這麼多 無辜 的 生命 不能 白白 犧牲 相關 新聞 台鐵 出包 旅客 買 了 自強號 看不到 座位 悲情 一路 站到 目的地 交通部 回應 表示 林佳龍 已 要求 台鐵 檢討 懲處 名單 報部 核定 提高 懲處 層級 包括 局長 在內 一併 全面 檢討 相關 責任 林佳龍 在 年初 上任 交通部長 後 要求 針對 普悠瑪 列車 翻覆 事故 重啟 調查 擴大 懲處 提高 賠償 家屬 陸續 接受 賠償 條件 據了解 今年 事故 周年 紀念 前夕 林佳龍 要求 台鐵局 鐵道局 路政司 等 單位 向 他 彙報 台鐵 的 安全 策進 作為 原是 希望 藉 此 激勵 台鐵 提高 安全 管理 豈料 卻 因 得知 事故 經過 而 大怒 運安會 曾 在 今年 月 日 發布 安全 通告 要求 台鐵 對 全台 的 彎道 鋼軌 加強 檢查 當時 曾 指出 台鐵 月 日 發生 內軌 斷裂 所幸 無 人 傷亡 但 並未 對外 透露 事發 詳細 經過 交通部 月 日 當天 雖 接獲 台鐵 簡訊 通報 有 鋼軌 事故 但 稍 晚 即已 緊急 修復 恢復 雙向 通行 不過 據 蘋果 掌握 消息 林佳龍 日前 主動 要求 台鐵 報告 時 台鐵 才 首度 向 交通部 坦言 今年 月 日 晚間 時 分 台鐵 第 次 司機員 向 成功 站 通報 鐵軌 碰一聲 很 大聲 好像 是 斷軌 站務 人員 通知 道班 後 共同 前往 查看 但 竟然 未依 向 調度員 通報 導致 後續 有 兩班 列車 包括 次 莒光號 次 普悠瑪 自強號 高速 通過 該 地點 直到 調度員 自行 從 無線電 群組 聽到 成功 站 的 通話 得知 有 鋼軌 斷裂 才 在 時 分 發布 行車 命令 封鎖 該線 改 單線 雙向 行車 過程 相當 驚險 隨後 成功 